# Age-structured ICU demand sampler

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from scipy.optimize import minimize 
from datetime import timedelta
import datetime as dt
from IPython.display import Image
import requests

In [2]:
name = 'BA'
p_SUS = 0.8932

## Data

### Brazil

#### COVID-19 DATA

Data source: [Brasil.IO](https://brasil.io/dataset/covid19/caso). Observe the database changes every day.



In [3]:
#url = "https://brasil.io/dataset/covid19/caso/?format=csv"
#filename = 'data/brazil_' + url.split("/")[-3] + '.csv'
#with open(filename, "wb") as f:
#    r = requests.get(url)
#    f.write(r.content)

To keep the same analysis, we keep the data basis from June 11.

In [4]:
filename = 'data/brazil_covid19_06_11.csv'

df = pd.read_csv(filename)
df.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-06-10,AC,Acrelândia,city,167,2,True,15256.0,1200013.0,1094.65128,0.0120
1,2020-06-10,AC,Assis Brasil,city,79,4,True,7417.0,1200054.0,1065.12067,0.0506
2,2020-06-10,AC,Brasiléia,city,176,5,True,26278.0,1200104.0,669.76178,0.0284
3,2020-06-10,AC,Bujari,city,90,1,True,10266.0,1200138.0,876.68030,0.0111
4,2020-06-10,AC,Capixaba,city,73,2,True,11733.0,1200179.0,622.17677,0.0274


In [5]:
state_or_city = 'state'
#state_or_city = 'city'

#name = 'RJ'
#name = 'São Paulo'

df = df[ df['place_type'] == state_or_city]
df = df[ df[state_or_city] == name ]

In [6]:
df[ ['date', 'confirmed', 'confirmed_per_100k_inhabitants'] ]

,date,confirmed,confirmed_per_100k_inhabitants
579,2020-06-10,32685,219.75969
4886,2020-06-09,30481,204.94096
9521,2020-06-08,28715,193.06714
14128,2020-06-07,28250,189.94069
18716,2020-06-06,27396,184.19876
...,...,...,...
187574,2020-03-11,3,0.02017
187589,2020-03-10,2,0.01345
187603,2020-03-09,2,0.01345
187626,2020-03-07,2,0.01345


In [7]:
df.groupby('date')[['confirmed', 'confirmed_per_100k_inhabitants']].sum()

,confirmed,confirmed_per_100k_inhabitants
date,,
2020-03-06,1,0.00672
2020-03-07,2,0.01345
2020-03-09,2,0.01345
2020-03-10,2,0.01345
2020-03-11,3,0.02017
...,...,...
2020-06-06,27396,184.19876
2020-06-07,28250,189.94069
2020-06-08,28715,193.06714


In [8]:
df.groupby('date')[['confirmed']].sum()

,confirmed
date,
2020-03-06,1
2020-03-07,2
2020-03-09,2
2020-03-10,2
2020-03-11,3
...,...
2020-06-06,27396
2020-06-07,28250
2020-06-08,28715


Select a particular state or city:

In [9]:
state_or_city = 'state'


df = df[ df['place_type'] == state_or_city]
df = df[ df[state_or_city] == name ]

df_I = df.groupby('date')[['confirmed', 'confirmed_per_100k_inhabitants']].sum()
df_I.index = pd.to_datetime(df_I.index)
today = df_I.index[-1] + timedelta(days=1)

In [10]:
df_I

,confirmed,confirmed_per_100k_inhabitants
date,,
2020-03-06,1,0.00672
2020-03-07,2,0.01345
2020-03-09,2,0.01345
2020-03-10,2,0.01345
2020-03-11,3,0.02017
...,...,...
2020-06-06,27396,184.19876
2020-06-07,28250,189.94069
2020-06-08,28715,193.06714


In [11]:
alpha = 30.
alpha2 = 7.

In [12]:
def delta(df_conf):

    list_ = []
    list_.append(0)

    for j in range(len(df_conf) - 1):

        list_.append(df_conf[j+1] - df_conf[j])

    return list_

In [13]:
df_I['growth_rate'] = delta(df_I['confirmed'])
# Exponential weight
df_I['growth_rate_EWM'] = df_I['growth_rate'].ewm(com= alpha).mean()
# Rate
df_I['growth_accel'] = delta(df_I['growth_rate_EWM'])
# Exponential weight
df_I['growth_accel_EWM'] = df_I['growth_accel'].ewm(com= alpha2).mean()


df_I['growth_rate_EWM_NORM_(%)'] = 100*df_I['growth_rate_EWM'] / df_I['confirmed']
df_I['growth_accel_EWM_NORM_(%)'] = 100*df_I['growth_accel_EWM'] / df_I['confirmed']

df_I['growth_rate_NORM_(%)'] = 100*df_I['growth_rate'] / df_I['confirmed']
df_I['growth_accel_NORM_(%)'] = 100*df_I['growth_accel'] / df_I['confirmed']

In [14]:
df_I

,confirmed,confirmed_per_100k_inhabitants,growth_rate,growth_rate_EWM,growth_accel,growth_accel_EWM,growth_rate_EWM_NORM_(%),growth_accel_EWM_NORM_(%),growth_rate_NORM_(%),growth_accel_NORM_(%)
date,,,,,,,,,,
2020-03-06,1,0.00672,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-07,2,0.01345,1,0.508197,0.508197,0.271038,25.409836,13.551913,50.000000,25.409836
2020-03-09,2,0.01345,0,0.333214,-0.174983,0.102131,16.660695,5.106543,0.000000,-8.749141
2020-03-10,2,0.01345,0,0.245770,-0.087444,0.044867,12.288475,2.243342,0.000000,-4.372220
2020-03-11,3,0.02017,1,0.406667,0.160897,0.074643,13.555552,2.488105,33.333333,5.363236
...,...,...,...,...,...,...,...,...,...,...
2020-06-06,27396,184.19876,977,608.969395,12.944462,24.948984,2.222841,0.091068,3.566214,0.047249
2020-06-07,28250,189.94069,854,617.280559,8.311163,22.869247,2.185064,0.080953,3.023009,0.029420
2020-06-08,28715,193.06714,465,612.123937,-5.156622,19.365999,2.131722,0.067442,1.619363,-0.017958


In [15]:
states_3 = ['BA']

state_or_city = 'state'
fit_until = '06-10'

for j in states_3:
    name = str(j)
    print(name)

    path = 'results/dfs/'
    path_ = 'results/dfs/csv/'

    file_ = 'df_ICU_' + state_or_city + '_' + name + '_fit_until_' + fit_until

    df = pd.read_pickle(path + file_ + '.pkl')
    #df.to_csv(path_ + file_ + '.csv', index= True, sep= ';')

BA


In [16]:
df

,n_mean,n_std,n_mean_ICU,n_std_ICU,n_mean_ICU_SUS,n_std_ICU_SUS,n_mean_ICU_PRIVATE,n_std_ICU_PRIVATE
date,,,,,,,,
2020-03-06,1.0,0.964941,0.00,0.964941,0.00,0.964941,0.00,1.376972
2020-03-07,2.0,1.364114,0.00,1.364114,0.00,1.364114,0.00,1.795807
2020-03-09,2.0,1.393346,0.00,1.393346,0.00,1.393346,0.00,1.826133
2020-03-10,2.0,1.390952,0.00,1.390952,0.00,1.390952,0.00,1.823650
2020-03-11,3.0,1.684870,0.00,1.684870,0.00,1.684870,0.00,2.126889
...,...,...,...,...,...,...,...,...
2020-06-06,27396.0,157.321528,875.62,197.055697,781.04,197.316411,94.58,197.555724
2020-06-07,28250.0,162.075128,879.53,203.592693,785.04,203.810311,94.49,204.092614
2020-06-08,28715.0,164.671367,883.75,204.398549,789.40,204.582469,94.35,204.898387


In [17]:
df2 = df.drop(columns=['n_mean', 'n_std', 'n_mean_ICU_PRIVATE', 'n_std_ICU_PRIVATE'])

In [20]:
df2.join(df_I)

,n_mean_ICU,n_std_ICU,n_mean_ICU_SUS,n_std_ICU_SUS,confirmed,confirmed_per_100k_inhabitants,growth_rate,growth_rate_EWM,growth_accel,growth_accel_EWM,growth_rate_EWM_NORM_(%),growth_accel_EWM_NORM_(%),growth_rate_NORM_(%),growth_accel_NORM_(%)
date,,,,,,,,,,,,,,
2020-03-06,0.00,0.964941,0.00,0.964941,1,0.00672,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-07,0.00,1.364114,0.00,1.364114,2,0.01345,1,0.508197,0.508197,0.271038,25.409836,13.551913,50.000000,25.409836
2020-03-09,0.00,1.393346,0.00,1.393346,2,0.01345,0,0.333214,-0.174983,0.102131,16.660695,5.106543,0.000000,-8.749141
2020-03-10,0.00,1.390952,0.00,1.390952,2,0.01345,0,0.245770,-0.087444,0.044867,12.288475,2.243342,0.000000,-4.372220
2020-03-11,0.00,1.684870,0.00,1.684870,3,0.02017,1,0.406667,0.160897,0.074643,13.555552,2.488105,33.333333,5.363236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-06,875.62,197.055697,781.04,197.316411,27396,184.19876,977,608.969395,12.944462,24.948984,2.222841,0.091068,3.566214,0.047249
2020-06-07,879.53,203.592693,785.04,203.810311,28250,189.94069,854,617.280559,8.311163,22.869247,2.185064,0.080953,3.023009,0.029420
2020-06-08,883.75,204.398549,789.40,204.582469,28715,193.06714,465,612.123937,-5.156622,19.365999,2.131722,0.067442,1.619363,-0.017958
